# Extract the Pettifor and Glawe scales from [Glawe et al. 2018](https://iopscience.iop.org/article/10.1088/1367-2630/18/9/093011/meta)

In [ ]:
import numpy as np
import pandas as pd

The data was extracted from the paper using [Tabula](https://tabula.technology/) and stored as csv

In [ ]:
df = pd.read_csv('data/tabula-Glawe2016.csv', header=None).T

In [ ]:
df

In [ ]:
dfs = np.split(df, [5, 10, 15], axis=1)

In [ ]:
newdfs = []
for df in dfs:
    df.columns = df.iloc[0]
    df.reindex(df.drop(0, inplace=True))
    newdfs.append(df)
data = pd.concat(newdfs, axis=0)

In [ ]:
data = data.set_index('n').dropna()

In [ ]:
data.index = data.index.astype(int)

In [ ]:
data.head(20)

## Update the `elements.db` database

In [ ]:
from mendeleev import get_session, Element
session = get_session()

## Pettifor number

In [ ]:
pn = data['P'].reset_index().set_index('P')

In [ ]:
pn.info()

In [ ]:
pn.head()

In [ ]:
for i, row in pn.iterrows():
    e = session.query(Element).filter(Element.symbol == i).one()
    print(i, e.symbol, row['n'], e.pettifor_number)
    e.pettifor_number = int(row['n'])
    session.add(e)
session.commit()

## Glawe number

In [ ]:
gn = data['Pm'].reset_index().set_index('Pm')

In [ ]:
gn.info()

In [ ]:
gn.head()

In [ ]:
for i, row in gn.iterrows():
    e = session.query(Element).filter(Element.symbol == i).one()
    print(i, e.symbol, row['n'], e.glawe_number)
    e.glawe_number = int(row['n'])
    session.add(e)
session.commit()

In [ ]:
session.close()